In [134]:
import os
import pickle
import pandas as pd
import numpy as np
import nbimporter
import preprocessing # import Jupyter notebook
#from preprocessing import clean_pdf

import gensim.corpora as corpora
import collections


In [140]:
def ids_to_keep(meta_df,fields):
    pass
def clean_pdf(text_df, file_name='', output_dir='',section_lvl = False):
    
    start = time.time()
    

    # if index is not paper_id
    if text_df.index.name is None: 
        print('changing index to paper_id')
        text_df = text_df.set_index('paper_id')

    ids = text_df.index.values.astype(str)
        
    contents = text_df['whole_text'].values.tolist()
    
    # Add abstract to text
    if not section_lvl:
        abstracts = text_df['abstract'].values.tolist()

        contents = [i + j for i, j in zip(contents, abstracts)]
    
    t = time.time()
    print(t-start)
    
    # Remove new line characters
    #print(contents)
    
    contents = map(lambda x: re.sub('\s+', ' ', x), contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: lower case text
    contents = map(lambda x: x.lower(),contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: keep alphanumeric
    contents = map(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x), contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: remove stand along numbers
    contents = map(lambda x: re.sub(" \d+ ", " ", x), contents)

    t = time.time()
    print(t-start)
    
    # Preprocessing: remove stop words
    contents = map(remove_stopwords, contents)
    
    t = time.time()
    print(t-start)
    
    contents = list(contents)

    # Preprocessing: remove short text
    inds = find_longer_text(contents)
    contents = itertools.compress(contents, inds)
    ids = list(itertools.compress(ids, inds))
    
    key_words = text_df.loc[ids]['key_words'].values
    print('Tokenizing')
    
    # Tokenize words + remove punctuation
    tokenized_contents = map(tokenize,contents) # documents in BOW format
#     word_list = [tokenize(article) for article in contents]
    t = time.time()
    print(t-start)
    
    # Remove numbers
    tokenized_contents = map(remove_digits, tokenized_contents)
    
    
    t = time.time()
    print(t-start)
    
    # Keep longer words
#     word_list = [keep_longer_words(words) for words in  word_list]
    tokenized_contents = map(keep_longer_words,  tokenized_contents)
    
    
    t = time.time()
    print(t-start)
    
    print('Lemmatizing')
    
    # Preprocessing: lemmatize
    tokenized_contents = map(lemmatize, tokenized_contents)
    
#     print(list(word_list)[0])
    
    t = time.time()
    print(t-start)
    
    print('Bag of Words Representation')
    tokenized_contents = list(tokenized_contents)

    dct = corpora.Dictionary(tokenized_contents) # make dct before corpus
#     doc2bow = partial(dct.doc2bow,allow_update=True)
    
    print('length of dct before filter_extreme: ', len(dct))
    dct.filter_extremes() # using default params # filter dct before creating corpus
    print('length of dct after filter_extreme: ', len(dct))
    

    # Make corpus after any changes to dct
    corpus = list(map(lambda x: dct.doc2bow(x,allow_update=True), tokenized_contents))    
#     corpus = [dct.doc2bow(doc, allow_update=True) for doc in word_list]

    t = time.time()
    print(t-start)


    
    word_list =  [item for sublist in tokenized_contents for item in sublist]
    counter=collections.Counter(word_list)
    
    d = {'dct': dct, 'corpus': corpus, 'docs': tokenized_contents, 
                 'counter': counter, 'ids': ids, 'word_list':word_list,
                 'key_words':key_words}
                 
    if file_name:
        os.makedirs(output_dir, exist_ok=True)
        output_file_name = output_dir + file_name + '_clean.pkl'


        with open(output_file_name, 'wb') as f:  # Python 3: open(..., 'wb') 
            pickle.dump(d, f)
    return d

In [159]:
# Load full dataset
with open('./Dataset/merged/textdata_all.pkl', 'rb') as f:
    text_df = pickle.load(f)
text_df.reset_index(drop=True, inplace=True)

In [160]:
text_df.head(5)

,paper_id,abstract,key_words,body_text,whole_text,citations
0,18980380,This technical note studies Markov decision pr...,"[Distributional robustness, Markov decision p...","[{'section': 'II. PRELIMINARIES', 'text': 'Thr...","Throughout the technical note, we use capital ...",{'BIBREF0': {'title': 'Distributionally robust...
1,56031008,Abstract-The present study attempted to find o...,"[listening comprehension, pre-task activities...",[{'section': 'A. Listening Materials and Activ...,Morley (1991) has explained that in developing...,"{'BIBREF0': {'title': 'Listening', 'authors': ..."
2,88484504,"In this paper, we address robust design of sym...","[Downlink MU-MISO, imperfect CSI, symbolleve...",[{'section': 'II. SYSTEM AND UNCERTAINTY MODEL...,We consider an MU-MISO wireless broadcast chan...,{'BIBREF0': {'title': 'Convex optimization-bas...
3,88485902,ABSTRACT A kinematic equation of profiling flo...,"[Profiling float, depth control, low power c...","[{'section': 'I. INTRODUCTION', 'text': 'With ...",With the increase in the cognition of marine a...,{'BIBREF0': {'title': 'AUV buoyancy regulating...
4,204197524,We characterize practical optical signal recei...,"[Optical wireless communications, multi-stage...","[{'section': 'A. PMT Principle Review', 'text'...",The typical structure of a PMT is shown in Fig...,{'BIBREF0': {'title': 'A statistical non-linea...


In [156]:
# Load full dataset
with open('./Dataset/text_df/pdf_parses_10.pkl', 'rb') as f:
    batch_df = pickle.load(f)
batch_df.reset_index(drop=True, inplace=True)

In [162]:
zero_df = text_df.iloc[whole_text_inds]
zero_df.shape

IndexError: Boolean index has wrong length: 13715 instead of 10090

In [163]:
whole_text_inds = [type(x) != str or len(x)== 0  for x in text_df['whole_text']]
print(np.sum(whole_text_inds), text_df.shape[0])


5625 10090


In [165]:
docs_cleaned = clean_pdf(text_df)

changing index to paper_id
0.1245725154876709
0.1245725154876709
0.1245725154876709
0.1245725154876709
0.1245725154876709
0.1245725154876709
Tokenizing
53.112943172454834
53.112943172454834
53.112943172454834
Lemmatizing
53.112943172454834
Bag of Words Representation
length of dct before filter_extreme:  158730
length of dct after filter_extreme:  25185
159.25587630271912


In [169]:
print(len(docs_cleaned['ids']), len(docs_cleaned['corpus']))

4443 4443


## Cleaning sections

In [59]:
def process_sections(textdf, file_name=''):
    def merge_sections(doc):
        sections = doc['body_text']
        #print(len(sections))
        #print(sections[0])
        if type(sections) is list and len(sections)>0:
            section_titles = [x['section'] for x in sections]
            paper_id = doc['paper_id']
            texts = []; titles = [];
            current_text = sections[0]['text']
            current_title = section_titles[0]
            for i in range(1,len(sections)):
                #print(i)
                if section_titles[i] == current_title:
                    current_text = current_text + ' ' + sections[i]['text']
                else: 
                    texts.append(current_text)
                    titles.append(current_title)
                    current_title = section_titles[i]
                    current_text = sections[i]['text']
            sections_df = pd.DataFrame({'whole_text': texts, 'titles':titles, 'paper_id':paper_id})
            return sections_df
        else:
            return []
    total_df = pd.DataFrame(columns = ['whole_text', 'titles', 'paper_id'])
    for index,row in textdf.iterrows():
        #print(index)
        
        total_df = total_df.append(merge_sections(row))
    return total_df

In [60]:
sections_df = process_sections(text_df)

In [61]:
sections_df.head(10)

,whole_text,titles,paper_id
0,"Throughout the technical note, we use capital ...",II. PRELIMINARIES,18980380
1,1) The confidence set O ns s is bounded and ha...,Assumption 3 (Regularity Conditions forC s ):,18980380
2,This section focuses on DAMDP with a finite nu...,III. FINITE HORIZON DISTRIBUTIONALLY ROBUST MDPS,18980380
3,"In this section, we study two synthetic numeri...",IV. SIMULATION,18980380
4,We consider a machine replacement problem simi...,A. Reward Uncertainty in the Machine Replaceme...,18980380
5,We first consider a machine replacement proble...,1) Machine Replacement as a MDP With Gaussian ...,18980380
6,The second experiment has a similar setup as t...,2) Machine Replacement as a MDP With Mixed Gau...,18980380
7,"We now consider a path planning problem, simil...",B. Transition Uncertainty in the Path Planning...,18980380
0,Morley (1991) has explained that in developing...,A. Listening Materials and Activities,56031008
1,"Prior knowledge in listener""s mind entails the...",B. Schema Theory and Background Knowledge,56031008


In [37]:
from nltk import pos_tag
from nltk.corpus import wordnet
# spacy for lemmatization
import spacy
import time
import re
import itertools 

def get_word_postag(word):
    #if pos_tag([word])[0][1].startswith('J'):
    #    return wordnet.ADJ
    #if pos_tag([word])[0][1].startswith('V'):
    #    return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        #return wordnet.NOUN
        return True
    else:
        return False
        #return wordnet.ADJ
        #return wordnet.NOUN

from nltk.tokenize import word_tokenize
# Preprocessing: tokenize words
def tokenize(text):
    return(word_tokenize(text))


def sent_to_words(sentences):
    for sentence in sentences:
        return(gensim.utils.simple_preprocess(str(sentence), min_len=3,deacc=True))  # deacc=True removes punctuations

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

# Preprocessing: remove stopwords
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords]) 
    #return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Preprocessing: lemmatizing
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Preprocessing: remove short text
def find_longer_text(texts,k=200):
    return list(map(lambda x: len(x.split())>k,texts))
    
#     lengths = list(map(lambda x: len(x.split()), texts))
#     return [val >= k for val in lengths]
    #return [idx for idx, val in enumerate(lengths) if val >= k] 

# Preprocessing: alpha num
def keep_alphanum(words):
    #def isalphanum(word):
    #return word.isalnum()
    return filter(lambda word: word.isalnum(), words)
    #return [word for word in words if word.isalnum()]

# Preprocessing: keep nouns
def keep_nouns(words):
    return filter(get_word_postag, words)
    #return [word for word in words if get_word_postag(word) =='n']

# Preprocessing: keep words >= 3 in length
def keep_longer_words(words):
    return list(filter(lambda x: (len(x) >= 3), words))
    #return [word for word in words if len(word) >= 3]

# Preprocessing: lemmatize
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
def lemmatize(words):
    return list(map(lm.lemmatize, words)) 

# Preprocessing: stemming
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 
def stemming(words):
    #return [ps.stem(word) for word in words]
    return map(ps.stem, words)

def remove_digits(words):
    return list(filter(lambda x: x.isalpha(), words))
    #return list(filter(lambda x: x.isalpha(), words))
#     return [word for word in words if word.isalpha()]

def merged(words):
    return ' '.join(word for word in words)        


In [43]:
def ids_to_keep(meta_df,fields):
    pass
def clean_section(text_df, file_name='', output_dir='',section_lvl = False):
    
    start = time.time()
    

    # if index is not paper_id
    if text_df.index.name is None: 
        print('changing index to paper_id')
        text_df = text_df.set_index('paper_id')

    ids = text_df.index.values.astype(str)
        
    contents = text_df['whole_text'].values.tolist()
    
    # Add abstract to text
    if not section_lvl:
        abstracts = text_df['abstract'].values.tolist()

        contents = [i + j for i, j in zip(contents, abstracts)]
    
    t = time.time()
    print(t-start)
    
    # Remove new line characters
    contents = map(lambda x: re.sub('\s+', ' ', x), contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: lower case text
    contents = map(lambda x: x.lower(),contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: keep alphanumeric
    contents = map(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x), contents)
    
    t = time.time()
    print(t-start)
    
    # Preprocessing: remove stand along numbers
    contents = map(lambda x: re.sub(" \d+ ", " ", x), contents)

    t = time.time()
    print(t-start)
    
    # Preprocessing: remove stop words
    contents = map(remove_stopwords, contents)
    
    t = time.time()
    print(t-start)
    
    contents = list(contents)

    # Preprocessing: remove short text
    inds = find_longer_text(contents)
    contents = itertools.compress(contents, inds)
    ids = list(itertools.compress(ids, inds))
    
    #key_words = text_df.loc[ids]['key_words'].values
    print('Tokenizing')
    
    # Tokenize words + remove punctuation
    tokenized_contents = map(tokenize,contents) # documents in BOW format
#     word_list = [tokenize(article) for article in contents]
    t = time.time()
    print(t-start)
    
    # Remove numbers
    tokenized_contents = map(remove_digits, tokenized_contents)
    
    
    t = time.time()
    print(t-start)
    
    # Keep longer words
#     word_list = [keep_longer_words(words) for words in  word_list]
    tokenized_contents = map(keep_longer_words,  tokenized_contents)
    
    
    t = time.time()
    print(t-start)
    
    print('Lemmatizing')
    
    # Preprocessing: lemmatize
    tokenized_contents = map(lemmatize, tokenized_contents)
    
#     print(list(word_list)[0])
    
    t = time.time()
    print(t-start)
    
    print('Bag of Words Representation')
    tokenized_contents = list(tokenized_contents)

    dct = corpora.Dictionary(tokenized_contents) # make dct before corpus
#     doc2bow = partial(dct.doc2bow,allow_update=True)
    
    print('length of dct before filter_extreme: ', len(dct))
    dct.filter_extremes() # using default params # filter dct before creating corpus
    print('length of dct after filter_extreme: ', len(dct))
    

    # Make corpus after any changes to dct
    corpus = list(map(lambda x: dct.doc2bow(x,allow_update=True), tokenized_contents))    
#     corpus = [dct.doc2bow(doc, allow_update=True) for doc in word_list]

    t = time.time()
    print(t-start)


    
    word_list =  [item for sublist in tokenized_contents for item in sublist]
    counter=collections.Counter(word_list)
    
    if file_name:
        os.makedirs(output_dir, exist_ok=True)
        output_file_name = output_dir + file_name + '_clean.pkl'


        with open(output_file_name, 'wb') as f:  # Python 3: open(..., 'wb') 
            d = {'dct': dct, 'corpus': corpus, 'docs': tokenized_contents, 
                 'counter': counter, 'ids': ids, 'word_list':word_list}
            pickle.dump(d, f)
    return d

In [48]:
sections_cleaned = clean_section(sections_df, file_name='section_level_kw', output_dir='./Dataset/cleaned/cs-med/',section_lvl = True)

changing index to paper_id
0.002969980239868164
0.002969980239868164
0.002969980239868164
0.002969980239868164
0.002969980239868164
0.002969980239868164
Tokenizing
5.570342540740967
5.570342540740967
5.570342540740967
Lemmatizing
5.570342540740967
Bag of Words Representation
length of dct before filter_extreme:  31656
length of dct after filter_extreme:  7942
13.497344732284546


In [ ]:
# Calculate weights

def ts_doc(dct, corpus, paper_ids):
    paper_ids_unique = np.unique(paper_ids)
    nwords = len(sections_cleaned['dct'].token2id)
    weights = np.zeros((nwords, len(sections_cleaned['corpus'])))
    for paper_id in paper_ids_unique:
        inds = np.where(paper_ids==paper_id)
        nonzero_words = []
        for ind in inds:
            nonzero_words_add = [corpus[ind][x][0] for x in range(len(corpus[ind]))]
            nonzero_words = nonzero_words + nonzero_words_add
        nonzero_words = np.unique(nonzero_words)
        for 
        
def ts_section()

In [174]:
dct = sections_cleaned['dct']
dct.cfs[0]

2360

In [177]:
sum([x[1] for x in corpus[0]])

392

In [ ]:
corpus = sections_cleaned['corpus']
dct = sections_cleaned['dct']

weights = np.zeros(())
for d_idx,d in enumerate(corpus):
    for w_idx, c in enumerate(d):
        
        # section length
        section_length = sum([x[1] for x in d])
        
        # Calculate section-level weight
        weight_section = (c[1] + 1) / (dct.cfs[w_idx] + section_length)
        
        # Calculate document-level weight
        weight_document = () / ()
        
        # Combined weight
        weight_combined = weight_selection*weight_document
    
        weights[d_idx][w_idx] = weight_combined


In [170]:
docs_cleaned['dct']

In [92]:
nonzero_words = [sections_cleaned['corpus'][0][x][0] for x in range(len(sections_cleaned['corpus'][0]))]

In [93]:
len(nonzero_words)

208

In [124]:
paper_ids = sections_df['paper_id']
paper_id = paper_ids.iloc[0]

In [115]:
corpus = sections_cleaned['corpus']
inds = np.where(paper_ids==paper_id)
corpus_select = [ corpus[index] for index in inds[0] ]

In [130]:
corpus_select = [ corpus[index] for index in inds[0] ]
len(corpus_select)

8

In [51]:
# Calculate weights
sections_cleaned['corpus'][0]

[(0, 1),
 (1, 3),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 10),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 2),
 (14, 2),
 (15, 2),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 2),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 2),
 (30, 1),
 (31, 2),
 (32, 6),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 9),
 (43, 2),
 (44, 3),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 10),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 2),
 (58, 4),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 2),
 (64, 1),
 (65, 7),
 (66, 3),
 (67, 3),
 (68, 2),
 (69, 2),
 (70, 1),
 (71, 1),
 (72, 3),
 (73, 1),
 (74, 2),
 (75, 1),
 (76, 1),
 (77, 2),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 3),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 3),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 2),
 (94, 1),
 (95, 2),
 (96, 1),
 (97, 2),
 (98, 1),
 (99, 1),
 (100, 1